Assess Spaceship Titanic with CredoAI - Lens
============================================

In [1]:
import pandas as pd
import joblib

import warnings
warnings.filterwarnings('ignore')

from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData
from credoai.evaluators import ModelFairness, Performance

# For this to work, you need to "File / Save and export notebook as... / Executable Script" the notebook
import Spaceship_Titanic_data

(train) Number of rows = 8693 and Number of cols = 14
(test) Number of rows = 4277 and Number of cols = 13


Load trained model
------------------


In [2]:
feature_engineering = Spaceship_Titanic_data.feature_engineering  # This is necessary to unpickle the model

with open('model.jlb', 'rb') as file:
    model = joblib.load(file)
    
processor = model.steps[0][1]
drop_target = model.steps[1][1]
classifier = model.steps[2][1]

fe_eng = processor.steps[0][1]
imputer = processor.steps[1][1]

Run a battery of evaluators over Age
------------------------------------

Prepare the train data for evaluation

In [3]:
transformed_data = processor.fit_transform(Spaceship_Titanic_data.train_data)
dropped_data = drop_target.fit_transform(transformed_data)

imputed_data = imputer.fit_transform(fe_eng.fit_transform(Spaceship_Titanic_data.train_data)) # Skip the scaler to get more readable results
ages = pd.qcut(imputed_data.Age, 10) # Quantize ages in 10 clusters from min age to max age

Wrap both the model and the data in CredoAI - Lens structures

In [4]:
credo_model = ClassificationModel(name="titanic_default_classifier",
                                  model_like=classifier)
credo_data = TabularData(
    name="titanic-default",
    X=dropped_data,
    y=transformed_data.Transported,
    sensitive_features=ages
)

Create a Lens and execute evaluators

In [5]:
lens = Lens(model=credo_model, assessment_data=credo_data)

# Precission score: % of true positives versus false positives https://credoai-lens.readthedocs.io/en/latest/pages/metrics.html#precision-score
# Recall score: probability of positive test if conditions are positive https://credoai-lens.readthedocs.io/en/latest/pages/metrics.html#true-positive-rate

metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))
lens.run()

2023-04-19 12:07:22,633 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: Age
2023-04-19 12:07:22,707 - lens - INFO - Evaluator Performance added to pipeline. 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished


In [14]:
fairness_results = lens.get_results(evaluator_name='ModelFairness')[0]

display(fairness_results['results'][0])
display(fairness_results['results'][1])

,type,value
0,equal_opportunity,0.009469
0,precision_score_parity,0.062779


,VIP,type,value
0,False,precision_score,0.790554
1,True,precision_score,0.853333


In [15]:
performance_results = lens.get_results(evaluator_name='Performance')[0]
# first dataframe is overall metrics
display(performance_results['results'][0])
# second dataframe is the long form of the confusion matrix
display(performance_results['results'][1])

IndexError: list index out of range

Run a simple evaluation over VIP
--------------------------------

Prepare the train data for evaluation

In [10]:
vip = imputed_data.VIP

Wrap both the model and the data in CredoAI - Lens structures

In [11]:
credo_model = ClassificationModel(name="titanic_default_classifier",
                                  model_like=classifier)
credo_data = TabularData(
    name="titanic-default",
    X=dropped_data,
    y=transformed_data.Transported,
    sensitive_features=vip
)

Create a Lens and execute evaluators

In [13]:
lens = Lens(model=credo_model, assessment_data=credo_data)

# Precission score: % of true positives versus false positives https://credoai-lens.readthedocs.io/en/latest/pages/metrics.html#precision-score
# Recall score: probability of positive test if conditions are positive https://credoai-lens.readthedocs.io/en/latest/pages/metrics.html#true-positive-rate

metrics = ['precision_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.run()

2023-04-19 12:50:33,586 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: VIP
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [16]:
fairness_results = lens.get_results(evaluator_name='ModelFairness')[0]

display(fairness_results['results'][0])
display(fairness_results['results'][1])

,type,value
0,equal_opportunity,0.009469
0,precision_score_parity,0.062779


,VIP,type,value
0,False,precision_score,0.790554
1,True,precision_score,0.853333
